In [1]:
import numpy as np
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup

In [2]:
BeautifulSoup("http://www.google.com")

/var/folders/ds/qp3gbx7n3tz0738b8w4wxs580000gn/T/ipykernel_28303/2709618310.py:1: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  BeautifulSoup("http://www.google.com")


http://www.google.com

In [3]:
url = "https://httpbin.org/user-agent"
r = requests.get(url)
myjson = json.loads(r.text)
useragent = myjson["user-agent"]
headers = {"User-Agent": useragent, "From": "mdg7wj@virginia.edu"}

In [11]:
url = "https://www.rottentomatoes.com/browse/movies_in_theaters/sort:a_z?page=5"
r = requests.get(url, headers=headers)
r

<Response [200]>

In [12]:
soup = BeautifulSoup(r.text, "html.parser")
#soup

In [ ]:
#Get list of hyperlinks to individual pages

In [13]:
hreflist = soup.find_all("a", attrs = {"data-track":"scores"})

In [14]:
hreflist[3]["href"]

'/m/bad_boys_ride_or_die'

In [16]:
movielinks = ["https://www.rottentomatoes.com" + x["href"] for x in hreflist]

In [ ]:
#Extract data from one movie page

In [20]:
url = movielinks[3]
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text, "html.parser")

In [32]:
moviedata = {}

In [33]:
moviedata["title"] = soup.find("title").text.replace(" | Rotten Tomatoes", "")

In [38]:
moviedata

{'title': 'Bad Boys: Ride or Die',
 'description': "This Summer, the world's favorite Bad Boys are back with their iconic mix of edge-of-your seat action and outrageous comedy but this time with a twist: Miami's finest are now on the run.",
 'critic_score': '64%',
 'audience_score': '97%',
 'critics_consensus': "Will Smith and Martin Lawrence remain good company even when Bad Boys strains to up the ante, proving there's still life left in this high-octane franchise."}

In [34]:
moviedata["description"] = soup.find("meta", attrs ={"name":"description"})['content']

In [35]:
moviedata["critic_score"] = soup.find("rt-button", slot = "criticsScore").text.strip()
moviedata["audience_score"] = soup.find("rt-button", slot = "audienceScore").text.strip()

In [37]:
moviedata["critics_consensus"] = soup.find("div", "consensus").p.text

In [46]:
cat = soup.find_all("div", "category-wrap")
cats = [x.text.strip().split("\n\n\n") for x in cat]
for c in cats:
    moviedata[c[0]] = c[1].replace("/n", "")

In [47]:
moviedata

{'title': 'Bad Boys: Ride or Die',
 'description': "This Summer, the world's favorite Bad Boys are back with their iconic mix of edge-of-your seat action and outrageous comedy but this time with a twist: Miami's finest are now on the run.",
 'critic_score': '64%',
 'audience_score': '97%',
 'critics_consensus': "Will Smith and Martin Lawrence remain good company even when Bad Boys strains to up the ante, proving there's still life left in this high-octane franchise.",
 'Director': 'Adil El Arbi, \nBilall Fallah',
 'Producer': 'Jerry Bruckheimer, \nWill Smith, \nChad Oman, \nDoug Belgrad',
 'Screenwriter': 'Chris Bremner',
 'Distributor': 'Columbia Pictures',
 'Production Co': 'Columbia Pictures, \nWestbrook Studios, \nJerry Bruckheimer Films',
 'Rating': 'R (Some Sexual References|Language Throughout|Strong Violence)',
 'Genre': 'Action, \nComedy',
 'Original Language': 'English',
 'Release Date (Theaters)': 'Jun 7, 2024, Wide',
 'Box Office (Gross USA)': '$177.5M',
 'Runtime': '1h 55m

In [49]:
def moviescrape(url):
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "html.parser")
    moviedata = {}
    
    moviedata["title"] = soup.find("title").text.replace(" | Rotten Tomatoes", "")
    moviedata["description"] = soup.find("meta", attrs ={"name":"description"})['content']
    moviedata["critic_score"] = soup.find("rt-button", slot = "criticsScore").text.strip()
    moviedata["audience_score"] = soup.find("rt-button", slot = "audienceScore").text.strip()
    try:
        moviedata["critics_consensus"] = soup.find("div", "consensus").p.text
    except:
        moviedata["critics_consensus"] = ""
    cat = soup.find_all("div", "category-wrap")
    cats = [x.text.strip().split("\n\n\n") for x in cat]
    for c in cats:
        moviedata[c[0]] = c[1].replace("/n", "")

    return moviedata

In [50]:
rt_data = [moviescrape(x) for x in movielinks]

In [56]:
rt_df = pd.DataFrame.from_records(rt_data)

In [57]:
rt_df

,title,description,critic_score,audience_score,critics_consensus,Director,Producer,Screenwriter,Distributor,Production Co,Genre,Original Language,Release Date (Theaters),Release Date (Streaming),Runtime,Rating,Box Office (Gross USA),Sound Mix,Aspect Ratio,Rerelease Date (Theaters)
0,A Man of Reason,After 10 years in the prison on behalf of his ...,14%,15%,,Jung Woo-sung,Song Dae-chan,"Jung Hae-sin, \nJung Woo-sung",Epic Pictures,Studio Take,"Action, \nDrama, \nMystery & Thriller",Korean,"Jul 5, 2024, Limited","Jul 9, 2024",1h 37m,NaN,NaN,NaN,NaN,NaN
1,A Quiet Place: Day One,Experience the day the world went quiet.,87%,73%,Grounded in raw humanity by Lupita Nyong'o and...,Michael Sarnoski,"Michael Bay, \nAndrew Form, \nBrad Fuller, \nJ...",Michael Sarnoski,Paramount Pictures,"Platinum Dunes, \nSunday Night","Horror, \nSci-Fi, \nMystery & Thriller",English,"Jun 28, 2024, Wide",NaN,1h 40m,PG-13 (Violent Content/Bloody Images|Terror),$94.0M,"Dolby Atmos, \nDolby Digital",Digital 2.39:1,NaN
2,Babes,BABES follows inseparable childhood friends Ed...,91%,79%,Offering an uproariously unvarnished look at m...,Pamela Adlon,"Ilana Glazer, \nJosh Rabinowitz, \nSusie Fox, ...","Ilana Glazer, \nJosh Rabinowitz",Neon,"FilmNation Entertainment, \nStarrpix, \nRange ...",Comedy,English,"May 24, 2024, Wide","Jul 2, 2024",1h 44m,R (Language Throughout|Some Drug Use|Sexual Ma...,$3.7M,NaN,NaN,NaN
3,Bad Boys: Ride or Die,"This Summer, the world's favorite Bad Boys are...",64%,97%,Will Smith and Martin Lawrence remain good com...,"Adil El Arbi, \nBilall Fallah","Jerry Bruckheimer, \nWill Smith, \nChad Oman, ...",Chris Bremner,Columbia Pictures,"Columbia Pictures, \nWestbrook Studios, \nJerr...","Action, \nComedy",English,"Jun 7, 2024, Wide",NaN,1h 55m,R (Some Sexual References|Language Throughout|...,$177.5M,"Dolby Atmos, \nDolby Digital",Digital 2.39:1,NaN
4,Banel & Adama,Banel and Adama are fiercely in love. The youn...,95%,,,Ramata-Toulaye Sy,"Éric Névé, \nMaud Leclair, \nAndrey Samoute Di...",Ramata-Toulaye Sy,Kino Lorber,"La Chauve Souris, \narte France Cinéma, \nAsto...",Drama,French (France),"Jun 7, 2024, Limited","Aug 6, 2024",1h 27m,NaN,$38.5K,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,The Strangers: Chapter 1,After their car breaks down in an eerie small ...,21%,44%,Lacking the chilling suspense of the original ...,Renny Harlin,"Courtney Solomon, \nMark Canton, \nChristopher...","Alan R. Cohen, \nAlan Freedland, \nBryan Bertino",Lionsgate,Fifth Element Productions,"Horror, \nMystery & Thriller",English,"May 17, 2024, Wide","Jun 7, 2024",1h 31m,R (Language|Horror Violence|Brief Drug Use),$35.1M,NaN,Digital 2.39:1,NaN
72,The Watchers (2024),"The Watchers follows Mina, a 28-year old artis...",32%,52%,Heavy on eerie mood but undone by an obtuse sc...,Ishana Shyamalan,"M. Night Shyamalan, \nAshwin Rajan, \nNimitt M...",Ishana Shyamalan,Warner Bros. Pictures,"New Line Cinema, \nInimitable Pictures, \nBlin...","Horror, \nMystery & Thriller",English,"Jun 7, 2024, Wide","Jun 28, 2024",1h 42m,PG-13 (Violence|Terror|Some Thematic Elements),$19.1M,Dolby Digital,Flat (1.85:1),NaN
73,Thelma (2024),Inspired by a real-life experience of director...,99%,83%,A stellar showcase for the talented June Squib...,Josh Margolin,"Zoë Worth, \nChris Kaye, \nNicholas Weinstock,...",Josh Margolin,Magnolia Pictures,"Bandwagon, \nZurich Avenue, \nInvention Studios","Comedy, \nAction",English,"Jun 21, 2024, Wide",NaN,1h 37m,PG-13 (Strong Language),$6.9M,NaN,NaN,NaN
74,Touch (2024),A romantic and thrilling story that spans seve...,100%,,,Baltasar Kormákur,"Baltasar Kormákur, \nAgnes Johansen, \nMike Go...","Baltasar Kormákur, \nÓlaf Ólafsson",Focus Features,"RVK Studios, \nGood Chaos","Romance, \nDrama",English,"Jul 12, 2024, Limited",NaN,2h 1m,R (Some Sexuality),NaN,NaN,NaN,NaN
